In [1]:
import requests
import os
import json

# API base URL and token
BASE_URL = "https://renforce.esapro.it/api"
API_TOKEN = "2c23937283ed6125f52c4ba7399c7baebdfb3f051ac9272550847066011d7d5a83807bb0c3c807e2450fcb7fd4843d126badcf4c3f0ee52ca548b43db77e100e48becf05cb5e93529564055dd8a85d9beb089420a91eec2a022fbf8f984d9138a26afc5296e60aed14024c60ddcb95dff130b3f9fb7af13598a48dbc62348389"

# Plant code
PLANT_CODE = "P1130"
RESOLUTION=0

# Headers for authentication
HEADERS = {
    "X-API-Token": API_TOKEN
}

# Function to retrieve plant configurations
def get_plant_configurations(plant_code):
    """
    Retrieves the plant configurations from the API.

    Args:
        plant_code (str): The plant code.

    Returns:
        dict: The API response as a dictionary.
    """
    url = f"{BASE_URL}/{plant_code}/config"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to retrieve configurations: {response.status_code}, {response.text}")

# Main execution
try:
    # Retrieve configurations
    configurations = get_plant_configurations(PLANT_CODE)

    # Print the response on the screen with better formatting
    print("\nAPI Response:")
    print("=" * 40)
    for config in configurations:
        for key, value in config.items():
            print(f"{key}: {value}")
        print("-" * 40)

    # Save the response to a file
    base_dir = "Data"
    os.makedirs(base_dir, exist_ok=True)
    file_path = os.path.join(base_dir, f"{PLANT_CODE}_configurations.json")
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(configurations, file, indent=4)
    print(f"\nConfigurations saved to: {file_path}")

    # Create a constant variable for the version
    VERSION = configurations[-1].get("v", "N/A")  # Assuming the version is in the first configuration
    print(f"\nVERSION constant created: {VERSION}")

except Exception as e:
    print("Error:", e)


API Response:
v: 1
t: 1980-01-01 00:00:00
----------------------------------------

Configurations saved to: Data/P1130_configurations.json

VERSION constant created: 1


In [2]:
import requests
import os
import csv
import json

def get_blocks(plant_code, config_version, block_types=None):
    """
    Retrieves a list of blocks (devices) from a specific plant configuration.

    Args:
        plant_code (str): The plant code (e.g., "P1234").
        config_version (int): Configuration identifier.
        block_types (str or list, optional): Type(s) of blocks to filter (e.g., "INVERTER" or ["COUNTER", "INVERTER"]).

    Returns:
        list: Array of block objects representing devices in the plant.
    """
    url = f"{BASE_URL}/{plant_code}/blocks/{config_version}"
    params = {}
    if block_types:
        params["type"] = block_types if isinstance(block_types, str) else ",".join(block_types)
    
    try:
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print("Response Status Code:", response.status_code)
            print("Response Text:", response.text)
            raise Exception(f"Failed to retrieve blocks: {response.status_code}, {response.text}")
    except requests.exceptions.RequestException as e:
        raise Exception(f"An error occurred while making the request: {e}")

def save_blocks_to_csv(plant_code, blocks):
    """
    Saves the blocks to a CSV file in the specified directory structure.

    Args:
        plant_code (str): The plant code.
        blocks (list): List of block objects.
    """
    # Define the directory structure
    base_dir = "Data"
    plant_dir = os.path.join(base_dir, plant_code)

    # Create directories if they don't exist
    os.makedirs(plant_dir, exist_ok=True)

    # Define the CSV file path
    csv_file_path = os.path.join(plant_dir, f"{plant_code}_blocks.csv")

    # Write blocks to the CSV file
    with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        # Write the header
        writer.writerow(["ID", "Name", "Type", "Description", "OID", "Properties", "Sensors", "Visible", "Enabled"])
        # Write the data
        for block in blocks:
            writer.writerow([
                block.get("id", "N/A"),
                block.get("name", "N/A"),
                block.get("type", "N/A"),
                block.get("desc", "N/A"),
                block.get("oid", "N/A"),
                block.get("props", "N/A"),
                block.get("sens", "N/A"),
                block.get("visible", "N/A"),
                block.get("enabled", "N/A")
            ])

    print(f"Blocks saved to: {csv_file_path}")

def print_blocks(blocks):
    """
    Prints the blocks nicely on the screen.

    Args:
        blocks (list): List of block objects.
    """
    print("\nBlocks List:")
    for block in blocks:
        print(f"ID: {block.get('id', 'N/A')}")
        print(f"Name: {block.get('name', 'N/A')}")
        print(f"Type: {block.get('type', 'N/A')}")
        print(f"Description: {block.get('desc', 'N/A')}")
        print(f"OID: {block.get('oid', 'N/A')}")
        print(f"Properties: {block.get('props', 'N/A')}")
        print(f"Sensors: {block.get('sens', 'N/A')}")
        print(f"Visible: {block.get('visible', 'N/A')}")
        print(f"Enabled: {block.get('enabled', 'N/A')}")
        print("-" * 40)  # Separator for readability

# Main execution
try:
    # Retrieve blocks
    blocks = get_blocks(PLANT_CODE, VERSION)

    # Print blocks nicely
    print_blocks(blocks)

    # Save blocks to CSV
    save_blocks_to_csv(PLANT_CODE, blocks)

except Exception as e:
    print("Error:", e)


Blocks List:
ID: 1
Name: P1130
Type: PLANT
Description: P1130 Collection
OID: 1
Properties: {'alarm_check_timerange': '10:00 - 14:00', 'alarm_irradiance_threshold': '100', 'avoid_night_oos': '0', 'capacity': '4103.2', 'contractual_irr_link': '', 'delivery_period': '30', 'energy_alerts': '0', 'field_alarm_ttl': '10', 'field_data_ttl': '10', 'irr_link.0': '171', 'sunrise_o': '0', 'sunset_o': '0', 'sys_loss': '0.05', 'temp_link.0': '173', 'up_inverter_thld': '0.9', 'up_irr_thld': '600', 'up_irrsd_thld': '0.05', 'up_jbox_thld': '0.9', 'user_irr_link': '', 'user_temp_link': ''}
Sensors: ['J44', 'J30', 'J101', 'J7', 'J34', 'J33', 'J31', 'J1', 'J104', 'J4', 'J102']
Visible: True
Enabled: True
----------------------------------------
ID: 17
Name: Cabina 01
Type: AGGREGATE
Description: 
OID: 1.1
Properties: {'aggr_stop': '1', 'alarm_irradiance_filter': '0', 'capacity': '1370.8', 'context': '', 'irr_link.0': '171', 'minor_aggr': '0', 'temp_link.0': '173', 'user_irr_link': '', 'user_temp_link': 

In [3]:
import requests
import os
import csv


def get_sensors(plant_code, config_version, sensor_types=None):
    """
    Retrieves a list of sensors from a specific plant configuration.

    Args:
        plant_code (str): The plant code (e.g., "P1234").
        config_version (int): Configuration identifier.
        sensor_types (str or list, optional): Type(s) of sensors to filter (e.g., "AC_ACTIVE_ENERGY" or ["AC_ACTIVE_ENERGY", "MODULE_SOLAR_IRRADIANCE"]).

    Returns:
        list: Array of sensor objects.
    """
    url = f"{BASE_URL}/{plant_code}/sensors/{config_version}"
    params = {}
    if sensor_types:
        params["type"] = sensor_types if isinstance(sensor_types, str) else ",".join(sensor_types)
    
    try:
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print("Response Status Code:", response.status_code)
            print("Response Text:", response.text)
            raise Exception(f"Failed to retrieve sensors: {response.status_code}, {response.text}")
    except requests.exceptions.RequestException as e:
        raise Exception(f"An error occurred while making the request: {e}")

def save_sensors_to_csv(plant_code, sensors):
    """
    Saves the sensors to a CSV file in the specified directory structure.

    Args:
        plant_code (str): The plant code.
        sensors (list): List of sensor objects.
    """
    # Define the directory structure
    base_dir = "Data"
    plant_dir = os.path.join(base_dir, plant_code)

    # Create directories if they don't exist
    os.makedirs(plant_dir, exist_ok=True)

    # Define the CSV file path
    csv_file_path = os.path.join(plant_dir, f"{plant_code}_sensors.csv")

    # Write sensors to the CSV file
    with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        # Write the header
        writer.writerow(["ID", "Name", "Type", "Description", "OID", "Properties", "Source", "Visible", "Enabled"])
        # Write the data
        for sensor in sensors:
            writer.writerow([
                sensor.get("id", "N/A"),
                sensor.get("name", "N/A"),
                sensor.get("type", "N/A"),
                sensor.get("desc", "N/A"),
                sensor.get("oid", "N/A"),
                sensor.get("props", "N/A"),
                sensor.get("source", "N/A"),
                sensor.get("visible", "N/A"),
                sensor.get("enabled", "N/A")
            ])

    print(f"Sensors saved to: {csv_file_path}")

def print_sensors(sensors):
    """
    Prints the sensors nicely on the screen.

    Args:
        sensors (list): List of sensor objects.
    """
    print("\nSensors List:")
    for sensor in sensors:
        print(f"ID: {sensor.get('id', 'N/A')}")
        print(f"Name: {sensor.get('name', 'N/A')}")
        print(f"Type: {sensor.get('type', 'N/A')}")
        print(f"Description: {sensor.get('desc', 'N/A')}")
        print(f"OID: {sensor.get('oid', 'N/A')}")
        print(f"Properties: {sensor.get('props', 'N/A')}")
        print(f"Source: {sensor.get('source', 'N/A')}")
        print(f"Visible: {sensor.get('visible', 'N/A')}")
        print(f"Enabled: {sensor.get('enabled', 'N/A')}")
        print("-" * 40)  # Separator for readability

# Main execution
try:
    plant_code = PLANT_CODE  # Plant code
    config_version = VERSION       # Configuration version
    sensor_types = None      # Retrieve all sensor types

    # Retrieve sensors
    sensors = get_sensors(plant_code, config_version, sensor_types)
    print(len(sensors), "sensors found")
    # Print sensors nicely
    print_sensors(sensors)

    # Save sensors to CSV
    save_sensors_to_csv(plant_code, sensors)
except Exception as e:
    print("Error:", e)

3572 sensors found

Sensors List:
ID: 1
Name: Voltage L1
Type: VOLTAGE
Description: 
OID: 1.1.14:1
Properties: {'alarm_c': '5', 'alarm_debounce': '60', 'alarm_function': '', 'alarm_max_threshold': '0', 'alarm_min_threshold': '0', 'alarm_tdc': '60', 'alarm_tdo': '60', 'alarm_type': '', 'cast': 'U32', 'endian': 'BE', 'is_raw': '1', 'limit_max': '', 'limit_min': '', 'register': '3c552', 'sampling_period': '300'}
Source: sundra
Visible: True
Enabled: True
----------------------------------------
ID: 10
Name: Temperature
Type: TEMPERATURE
Description: 
OID: 1.1.1.1:10
Properties: {'alarm_c': '3', 'alarm_debounce': '300', 'alarm_function': 'LIMITALARM', 'alarm_max_threshold': '70', 'alarm_min_threshold': '-10', 'alarm_tdc': '300', 'alarm_tdo': '300', 'alarm_type': '52', 'c_high_pass': '', 'c_low_pass': '', 'cast': 'U16', 'copy_to': '', 'data_age_lmt': '0', 'endian': 'BE', 'is_raw': '1', 'limit_max': '', 'limit_min': '', 'mask': '', 'mask_value': '1', 'register': '31064', 'sampling_period': '

In [4]:
import pandas as pd


# Path to the sensors file
SENSORS_FILE = f"Data/{PLANT_CODE}/{PLANT_CODE}_sensors.csv"

# Step 1: Load the sensors file
try:
    sensors_metadata = pd.read_csv(SENSORS_FILE)
except FileNotFoundError:
    print(f"File not found: {SENSORS_FILE}")
    exit()  # Ensure the program exits here to avoid further execution

# Step 2: Find the sensor with the name "Energia Prodotta" or "Energy AC DELTA"
if "Name" not in sensors_metadata.columns:
    print("The 'Name' column is missing in the sensors metadata.")
    exit()

sensor_row = sensors_metadata[sensors_metadata["Name"].str.contains("Energia Prodotta", case=False, na=False)]
if sensor_row.empty:
    sensor_row = sensors_metadata[sensors_metadata["Name"].str.contains("Energy AC DELTA", case=False, na=False)]
    if sensor_row.empty:
        print("Sensor with the name 'Energia Prodotta' or 'Energy AC DELTA' not found.")
        exit()

sensor_id = sensor_row.iloc[0]["ID"]
print(f"Sensor ID for the found sensor: {sensor_id}")

Sensor ID for the found sensor: 156


In [5]:
import pandas as pd
from datetime import datetime, timedelta
import requests
import time

# Path to the sensors file
SENSORS_FILE = f"Data/{PLANT_CODE}/{PLANT_CODE}_sensors.csv"

# Step 1: Load the sensors file
try:
    sensors_metadata = pd.read_csv(SENSORS_FILE)
except FileNotFoundError:
    print(f"File not found: {SENSORS_FILE}")
    exit()

# Step 2: Find the sensor with the name "Energia Prodotta" or "Energy AC DELTA"
if "Name" not in sensors_metadata.columns:
    print("The 'Name' column is missing in the sensors metadata.")
    exit()

sensor_row = sensors_metadata[sensors_metadata["Name"].str.contains("Energia Prodotta", case=False, na=False)]
if sensor_row.empty:
    sensor_row = sensors_metadata[sensors_metadata["Name"].str.contains("Energy AC DELTA", case=False, na=False)]
    if sensor_row.empty:
        print("Sensor with the name 'Energia Prodotta' or 'Energy AC DELTA' not found.")
        exit()

sensor_id = sensor_row.iloc[0]["ID"]
print(f"Sensor ID for the found sensor: {sensor_id}")

# Step 3: Retrieve data for the sensor
TREND_URL = f"{BASE_URL}/{PLANT_CODE}/data/trend/"
end_date = datetime.now().replace(minute=0, second=0, microsecond=0)  # Round down to the nearest full hour
max_interval = timedelta(days=92)  # Maximum allowed interval (92 days)
no_data_threshold = timedelta(days=365)  # Stop if no data is returned for more than 1 year

current_end = end_date
last_reading = None
no_data_duration = timedelta(0)

while no_data_duration < no_data_threshold:
    current_start = current_end - max_interval
    payload = {
        "from": current_start.strftime("%Y-%m-%d %H:%M:%S"),
        "to": current_end.strftime("%Y-%m-%d %H:%M:%S"),
        "sensors": [sensor_id],
        "resolution": RESOLUTION
    }

    # Make the POST request to the API
    response = requests.post(TREND_URL, headers=HEADERS, json=payload)

    if response.status_code == 200:
        try:
            data = response.json()
        except ValueError:
            print("Failed to parse JSON response.")
            break

        # Check if data is returned for the sensor
        if str(sensor_id) in data and data[str(sensor_id)]:
            sensor_data = data[str(sensor_id)]
            sensor_df = pd.DataFrame(sensor_data, columns=["timestamp", "value"])
            sensor_df["timestamp"] = pd.to_datetime(sensor_df["timestamp"])  # Ensure timestamp is datetime

            # Update the last reading to the minimum timestamp in this range
            last_reading = sensor_df["timestamp"].min()

            print(f"Data retrieved for range {current_start} to {current_end}.")
            no_data_duration = timedelta(0)  # Reset no data duration
            del sensor_df  # Clear memory
        else:
            print(f"No data for sensor {sensor_id} in range {current_start} to {current_end}.")
            no_data_duration += max_interval  # Increment no data duration
    else:
        print(f"Failed to fetch trend data. HTTP Status Code: {response.status_code}")
        print("Response:", response.text)
        break

    # Move to the previous time range
    current_end = current_start - timedelta(seconds=1)
    time.sleep(1)  # Avoid hitting API rate limits

# Step 4: Save the last reading going backward
if last_reading:
    last_day = last_reading.date()
    print(f"The last day of data for the sensor going backward is: {last_day}")
else:
    print("No data found for the sensor.")

Sensor ID for the found sensor: 156
Data retrieved for range 2025-03-10 23:00:00 to 2025-06-10 23:00:00.
Data retrieved for range 2024-12-08 22:59:59 to 2025-03-10 22:59:59.
Data retrieved for range 2024-09-07 22:59:58 to 2024-12-08 22:59:58.
Data retrieved for range 2024-06-07 22:59:57 to 2024-09-07 22:59:57.
Data retrieved for range 2024-03-07 22:59:56 to 2024-06-07 22:59:56.
Data retrieved for range 2023-12-06 22:59:55 to 2024-03-07 22:59:55.
Data retrieved for range 2023-09-05 22:59:54 to 2023-12-06 22:59:54.
Data retrieved for range 2023-06-05 22:59:53 to 2023-09-05 22:59:53.
Data retrieved for range 2023-03-05 22:59:52 to 2023-06-05 22:59:52.
Data retrieved for range 2022-12-03 22:59:51 to 2023-03-05 22:59:51.
Data retrieved for range 2022-09-02 22:59:50 to 2022-12-03 22:59:50.
Data retrieved for range 2022-06-02 22:59:49 to 2022-09-02 22:59:49.
Data retrieved for range 2022-03-02 22:59:48 to 2022-06-02 22:59:48.
No data for sensor 156 in range 2021-11-30 22:59:47 to 2022-03-02 2

In [6]:
# import requests
# import pandas as pd
# import time
# import os
# from datetime import datetime, timedelta

# # API endpoint URLs
# TREND_URL = f"{BASE_URL}/{PLANT_CODE}/data/trend/"
# SENSORS_URL = f"{BASE_URL}/{PLANT_CODE}/sensors/1"

# # Function to save sensors with data (ID and Name) to a CSV file
# def save_sensors_with_data(plant_code, sensors_with_data, sensors_metadata):
#     """
#     Saves the list of sensors with data (ID and Name) to a CSV file in the specified directory structure.

#     Args:
#         plant_code (str): The plant code.
#         sensors_with_data (list): List of sensor IDs with data.
#         sensors_metadata (pd.DataFrame): DataFrame containing sensor metadata.
#     """
#     # Define the directory structure
#     base_dir = "Data"
#     plant_dir = os.path.join(base_dir, plant_code)

#     # Create directories if they don't exist
#     os.makedirs(plant_dir, exist_ok=True)

#     # Define the CSV file path
#     csv_file_path = os.path.join(plant_dir, f"{plant_code}_sensors_with_data.csv")

#     # Filter metadata to include only sensors with data
#     matching_sensors = sensors_metadata[sensors_metadata["id"].isin(sensors_with_data)]  # Use "id" instead of "ID"

#     # Save the matching sensors (ID and Name) to the CSV file
#     matching_sensors[["id", "name"]].to_csv(csv_file_path, index=False)  # Use "id" and "name" columns

#     print(f"Sensors with data saved to: {csv_file_path}")

# # Fetch the list of sensors
# response_sensors = requests.get(SENSORS_URL, headers=HEADERS)

# if response_sensors.status_code == 200:
#     # Parse the JSON response to get the list of sensors
#     sensors_data = response_sensors.json()
#     sensors_metadata = pd.DataFrame(sensors_data)  # Convert sensor metadata to a DataFrame
#     sensor_ids = sensors_metadata["id"].tolist()  # Get all sensor IDs
#     print(f"Total sensor IDs: {len(sensor_ids)}")
    
#     # Use the last_day from the previous code as the start_date
#     try:
#         # Use the last_day from the previous code as the start_date
#         start_date = datetime.combine(last_day, datetime.min.time())  # Convert last_day (date) to datetime
#     except FileNotFoundError:
#         print("No previous last_day found. Using default start date.")
#         start_date = datetime(2025, 4, 15, 0, 0, 0)  # Default start date

#     # Define the end date
#     end_date = datetime(2025, 5, 15, 23, 59, 59)  # End date

#     # Define maximum time ranges for each resolution
#     resolution_max_intervals = {
#         0: timedelta(days=92),    # 15-minute resolution
#         1: timedelta(days=368),   # Hourly resolution
#         2: timedelta(days=7300),  # Daily resolution
#         3: timedelta(days=7300),  # Monthly resolution
#         4: timedelta(days=7300)   # Yearly resolution
#     }

#     # Get the maximum interval for the selected resolution
#     max_interval = resolution_max_intervals.get(RESOLUTION, timedelta(days=92))  # Default to 92 days if resolution is unknown

#     # Split the time range into chunks of the maximum interval
#     time_ranges = []
#     current_start = start_date
#     while current_start < end_date:
#         current_end = min(current_start + max_interval, end_date)
#         time_ranges.append((current_start, current_end))
#         current_start = current_end + timedelta(seconds=1)  # Avoid overlapping intervals

#     # Split sensor IDs into batches of 25 sensors per batch
#     batch_size = 25
#     sensor_batches = [sensor_ids[i:i + batch_size] for i in range(0, len(sensor_ids), batch_size)]

#     # Initialize an empty DataFrame
#     df = pd.DataFrame()

#     # List to track sensors with data
#     sensors_with_data = []

#     # Loop through each time range and batch of sensors
#     for time_idx, (start_time, end_time) in enumerate(time_ranges):
#         print(f"Processing time range {time_idx + 1}: {start_time} to {end_time}")
#         for batch_idx, batch in enumerate(sensor_batches):
#             print(f"Processing batch {batch_idx + 1} with sensors: {batch}")
#             payload = {
#                 "from": start_time.strftime("%Y-%m-%d %H:%M:%S"),
#                 "to": end_time.strftime("%Y-%m-%d %H:%M:%S"),
#                 "sensors": batch,
#                 "resolution": RESOLUTION  # Use the selected resolution
#             }

#             # Make the POST request to the API
#             response = requests.post(TREND_URL, headers=HEADERS, json=payload)

#             if response.status_code == 200:
#                 # Parse the JSON response
#                 data = response.json()

#                 # Track sensors with no data
#                 no_data_sensors = []

#                 # Transform the data into a DataFrame
#                 for sensor_id, sensor_data in data.items():
#                     if not sensor_data:
#                         no_data_sensors.append(sensor_id)  # Log sensors with no data
#                         continue
#                     sensors_with_data.append(sensor_id)  # Add sensor to the list of sensors with data
#                     sensor_df = pd.DataFrame(sensor_data, columns=["timestamp", sensor_id])
#                     sensor_df["timestamp"] = pd.to_datetime(sensor_df["timestamp"])  # Ensure timestamp is datetime
#                     if df.empty:
#                         df = sensor_df
#                     else:
#                         df = pd.merge(df, sensor_df, on="timestamp", how="outer")  # Merge on timestamp

#                 # Log sensors with no data for this batch
#                 if no_data_sensors:
#                     print(f"No data returned for sensors in batch {batch_idx + 1}: {', '.join(map(str, no_data_sensors))}")
#                 else:
#                     print(f"All sensors in batch {batch_idx + 1} have data.")
#             else:
#                 print(f"Failed to fetch trend data for batch {batch_idx + 1}. HTTP Status Code: {response.status_code}")
#                 print("Response:", response.text)

#             # Respect the API rate limit (no more than 5 calls per minute)
#             time.sleep(12)  # Wait 12 seconds between calls to stay under the limit

#     # Remove duplicates from the list of sensors with data
#     sensors_with_data = list(set(sensors_with_data))
#     # Display the DataFrame
#     print("DataFrame:")
#     print(df.head())
    
#     # Display the list of sensors with data
#     print("\nSensors with data:")
#     print(sensors_with_data)
#     print(f"Total sensors with data: {len(sensors_with_data)}")
    
#     # Save the list of sensors with data (ID and Name) to a CSV file
#     save_sensors_with_data(PLANT_CODE, sensors_with_data, sensors_metadata)

#     # Calculate and display the number of missing values
#     if not df.empty:
#         missing_values = df.isna().sum().sum()
#         print(f"Total missing values in the DataFrame: {missing_values}")

# else:
#     print(f"Failed to fetch sensors. HTTP Status Code: {response_sensors.status_code}")
#     print("Response:", response_sensors.text)

In [7]:
import requests
import pandas as pd
import time
import os
from datetime import datetime, timedelta

# API endpoint URLs
TREND_URL = f"{BASE_URL}/{PLANT_CODE}/data/trend/"
SENSORS_URL = f"{BASE_URL}/{PLANT_CODE}/sensors/1"

# Function to save sensors with data (ID and Name) to a CSV file
def save_sensors_with_data(plant_code, sensors_with_data, sensors_metadata):
    """
    Saves the list of sensors with data (ID and Name) to a CSV file in the specified directory structure.

    Args:
        plant_code (str): The plant code.
        sensors_with_data (list): List of sensor IDs with data.
        sensors_metadata (pd.DataFrame): DataFrame containing sensor metadata.
    """
    # Define the directory structure
    base_dir = "Data"
    plant_dir = os.path.join(base_dir, plant_code)

    # Create directories if they don't exist
    os.makedirs(plant_dir, exist_ok=True)

    # Define the CSV file path
    csv_file_path = os.path.join(plant_dir, f"{plant_code}_sensors_with_data.csv")

    # Filter metadata to include only sensors with data
    matching_sensors = sensors_metadata[sensors_metadata["id"].isin(sensors_with_data)]  # Use "id" instead of "ID"

    # Save the matching sensors (ID and Name) to the CSV file
    matching_sensors[["id", "name"]].to_csv(csv_file_path, index=False)  # Use "id" and "name" columns

    print(f"Sensors with data saved to: {csv_file_path}")

# Fetch the list of sensors
response_sensors = requests.get(SENSORS_URL, headers=HEADERS)

if response_sensors.status_code == 200:
    # Parse the JSON response to get the list of sensors
    sensors_data = response_sensors.json()
    sensors_metadata = pd.DataFrame(sensors_data)  # Convert sensor metadata to a DataFrame
    sensor_ids =  ["timestamp", "D.171.ISIRR", "172", "D.156.DELTA"]
    print(f"Total sensor IDs: {len(sensor_ids)}")
    
    # Use the last_day from the previous code as the start_date
    try:
        # Use the last_day from the previous code as the start_date
        start_date = datetime.combine(last_day, datetime.min.time())  # Convert last_day (date) to datetime
    except FileNotFoundError:
        print("No previous last_day found. Using default start date.")
        start_date = datetime(2025, 4, 15, 0, 0, 0)  # Default start date

    # Define the end date
    end_date = datetime(2025, 5, 15, 23, 59, 59)  # End date

    # Define maximum time ranges for each resolution
    resolution_max_intervals = {
        0: timedelta(days=92),    # 15-minute resolution
        1: timedelta(days=368),   # Hourly resolution
        2: timedelta(days=7300),  # Daily resolution
        3: timedelta(days=7300),  # Monthly resolution
        4: timedelta(days=7300)   # Yearly resolution
    }

    # Get the maximum interval for the selected resolution
    max_interval = resolution_max_intervals.get(RESOLUTION, timedelta(days=92))  # Default to 92 days if resolution is unknown

    # Split the time range into chunks of the maximum interval
    time_ranges = []
    current_start = start_date
    while current_start < end_date:
        current_end = min(current_start + max_interval, end_date)
        time_ranges.append((current_start, current_end))
        current_start = current_end + timedelta(seconds=1)  # Avoid overlapping intervals

    # Split sensor IDs into batches of 25 sensors per batch
    batch_size = 25
    sensor_batches = [sensor_ids[i:i + batch_size] for i in range(0, len(sensor_ids), batch_size)]

    # Initialize an empty DataFrame
    df = pd.DataFrame()

    # List to track sensors with data
    sensors_with_data = []

    # Loop through each time range and batch of sensors
    for time_idx, (start_time, end_time) in enumerate(time_ranges):
        print(f"Processing time range {time_idx + 1}: {start_time} to {end_time}")
        for batch_idx, batch in enumerate(sensor_batches):
            print(f"Processing batch {batch_idx + 1} with sensors: {batch}")
            payload = {
                "from": start_time.strftime("%Y-%m-%d %H:%M:%S"),
                "to": end_time.strftime("%Y-%m-%d %H:%M:%S"),
                "sensors": batch,
                "resolution": RESOLUTION  # Use the selected resolution
            }

            # Make the POST request to the API
            response = requests.post(TREND_URL, headers=HEADERS, json=payload)

            if response.status_code == 200:
                # Parse the JSON response
                data = response.json()

                # Track sensors with no data
                no_data_sensors = []

                # Transform the data into a DataFrame
                for sensor_id, sensor_data in data.items():
                    if not sensor_data:
                        no_data_sensors.append(sensor_id)  # Log sensors with no data
                        continue
                    sensors_with_data.append(sensor_id)  # Add sensor to the list of sensors with data
                    sensor_df = pd.DataFrame(sensor_data, columns=["timestamp", sensor_id])
                    sensor_df["timestamp"] = pd.to_datetime(sensor_df["timestamp"])  # Ensure timestamp is datetime
                    if df.empty:
                        df = sensor_df
                    else:
                        df = pd.merge(df, sensor_df, on="timestamp", how="outer")  # Merge on timestamp

                # Log sensors with no data for this batch
                if no_data_sensors:
                    print(f"No data returned for sensors in batch {batch_idx + 1}: {', '.join(map(str, no_data_sensors))}")
                else:
                    print(f"All sensors in batch {batch_idx + 1} have data.")
            else:
                print(f"Failed to fetch trend data for batch {batch_idx + 1}. HTTP Status Code: {response.status_code}")
                print("Response:", response.text)

            # Respect the API rate limit (no more than 5 calls per minute)
            time.sleep(12)  # Wait 12 seconds between calls to stay under the limit

    # Remove duplicates from the list of sensors with data
    sensors_with_data = list(set(sensors_with_data))
    # Display the DataFrame
    print("DataFrame:")
    print(df.head())
    
    # Display the list of sensors with data
    print("\nSensors with data:")
    print(sensors_with_data)
    print(f"Total sensors with data: {len(sensors_with_data)}")
    
    # Save the list of sensors with data (ID and Name) to a CSV file
    save_sensors_with_data(PLANT_CODE, sensors_with_data, sensors_metadata)

    # Calculate and display the number of missing values
    if not df.empty:
        missing_values = df.isna().sum().sum()
        print(f"Total missing values in the DataFrame: {missing_values}")

else:
    print(f"Failed to fetch sensors. HTTP Status Code: {response_sensors.status_code}")
    print("Response:", response_sensors.text)

Total sensor IDs: 4
Processing time range 1: 2022-03-18 00:00:00 to 2022-06-18 00:00:00
Processing batch 1 with sensors: ['timestamp', 'D.171.ISIRR', '172', 'D.156.DELTA']
All sensors in batch 1 have data.
Processing time range 2: 2022-06-18 00:00:01 to 2022-09-18 00:00:01
Processing batch 1 with sensors: ['timestamp', 'D.171.ISIRR', '172', 'D.156.DELTA']
All sensors in batch 1 have data.
Processing time range 3: 2022-09-18 00:00:02 to 2022-12-19 00:00:02
Processing batch 1 with sensors: ['timestamp', 'D.171.ISIRR', '172', 'D.156.DELTA']
All sensors in batch 1 have data.
Processing time range 4: 2022-12-19 00:00:03 to 2023-03-21 00:00:03
Processing batch 1 with sensors: ['timestamp', 'D.171.ISIRR', '172', 'D.156.DELTA']


MergeError: Passing 'suffixes' which cause duplicate columns {'172_x'} is not allowed.

In [31]:
df.head(20)

,timestamp,D.171.ISIRR_x,172_x,D.156.DELTA_x,172_y,D.156.DELTA_y,D.171.ISIRR_y
0,2022-03-18 07:00:00,152.901,2.529,NaN,NaN,NaN,NaN
1,2022-03-18 07:15:00,162.154,3.044,NaN,NaN,NaN,NaN
2,2022-03-18 07:30:00,174.519,3.237,NaN,NaN,NaN,NaN
3,2022-03-18 07:45:00,186.045,4.031,440.0,NaN,NaN,NaN
4,2022-03-18 08:00:00,196.427,3.915,772.0,NaN,NaN,NaN
5,2022-03-18 08:15:00,203.363,4.722,796.0,NaN,NaN,NaN
6,2022-03-18 08:30:00,214.244,4.848,908.0,NaN,NaN,NaN
7,2022-03-18 08:45:00,222.108,4.942,868.0,NaN,NaN,NaN
8,2022-03-18 09:00:00,221.688,5.528,784.0,NaN,NaN,NaN
9,2022-03-18 09:15:00,232.539,5.491,916.0,NaN,NaN,NaN


In [30]:
# Transform the data into a DataFrame
for sensor_id, sensor_data in data.items():
    if not sensor_data:
        no_data_sensors.append(sensor_id)  # Log sensors with no data
        continue
    sensors_with_data.append(sensor_id)  # Add sensor to the list of sensors with data
    sensor_df = pd.DataFrame(sensor_data, columns=["timestamp", sensor_id])
    sensor_df["timestamp"] = pd.to_datetime(sensor_df["timestamp"])  # Ensure timestamp is datetime
    sensor_df.set_index("timestamp", inplace=True)  # Set timestamp as the index
    if df.empty:
        df = sensor_df
    else:
        df = pd.merge(df, sensor_df, left_index=True, right_index=True, how="outer")  # Merge on index

/var/folders/f5/zvp8h0wn4d50th4gw1czk38m0000gn/T/ipykernel_50585/2962127736.py:13: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  df = pd.merge(df, sensor_df, left_index=True, right_index=True, how="outer")  # Merge on index
/var/folders/f5/zvp8h0wn4d50th4gw1czk38m0000gn/T/ipykernel_50585/2962127736.py:13: RuntimeWarning: '<' not supported between instances of 'Timestamp' and 'int', sort order is undefined for incomparable objects.
  df = pd.merge(df, sensor_df, left_index=True, right_index=True, how="outer")  # Merge on index


In [23]:
df.set_index("timestamp", inplace=True)

In [5]:
df

,Task,StartWeek,EndWeek,StartDate,EndDate
0,Define research problem and objectives,1,6,2025-05-22 21:39:40.090369,2025-06-26 21:39:40.090369
1,Conduct literature review,3,10,2025-06-05 21:39:40.090369,2025-07-24 21:39:40.090369
2,Collect & preprocess data,10,14,2025-07-24 21:39:40.090369,2025-08-21 21:39:40.090369
3,Explore & analyze data,12,15,2025-08-07 21:39:40.090369,2025-08-28 21:39:40.090369
4,Engineer features,14,16,2025-08-21 21:39:40.090369,2025-09-04 21:39:40.090369
5,"Select baseline models (ARIMA, SVR)",15,17,2025-08-28 21:39:40.090369,2025-09-11 21:39:40.090369
6,"Train ML models (RF, GBM, LSTM)",17,20,2025-09-11 21:39:40.090369,2025-10-02 21:39:40.090369
7,Integrate alarm data into models,18,21,2025-09-18 21:39:40.090369,2025-10-09 21:39:40.090369
8,Hybrid models w/ weather forecasts,20,23,2025-10-02 21:39:40.090369,2025-10-23 21:39:40.090369
9,"Compare model accuracy (RMSE, MAE)",21,24,2025-10-09 21:39:40.090369,2025-10-30 21:39:40.090369


In [29]:
df.to_csv(f"data_{PLANT_CODE}_{RESOLUTION}.csv")

In [ ]:
import pandas as pd
df = pd.read_csv('data_P1130_2.csv')

In [5]:
df.head()

,timestamp,100,1000,1011,1017,1021,1027,1031,1037,104,...,V941,V951,V961,V978,V979,V981,V993,V994,V996,V998
0,2022-03-18,27.719,1151989,20.146,26.649,19.875,29.930,19.522,32.877,21.287,...,3.067,3.067,3.067,284.574,0.039,102.094,288.152,0.049,94.763,102.462
1,2022-03-19,26.993,1152508,15.039,26.200,11.961,29.667,14.825,31.844,16.376,...,3.685,3.685,3.685,257.708,0.250,93.908,262.022,0.247,87.523,93.872
2,2022-03-20,26.148,1153129,17.980,26.230,15.384,29.474,17.485,31.637,16.777,...,3.853,3.853,3.853,306.430,0.146,95.131,307.965,0.144,87.641,93.898
3,2022-03-21,26.015,1153783,16.536,27.287,19.134,29.426,17.604,31.816,17.683,...,3.074,3.074,3.074,328.139,0.152,95.812,330.374,0.157,88.426,92.512
4,2022-03-22,26.456,1154462,18.330,27.309,18.249,29.787,17.626,32.235,19.207,...,3.543,3.543,3.543,340.998,0.277,97.367,342.816,0.273,89.729,96.263


In [13]:
df[["D.171.ISIRR","172","D.156.DELTA"]]

,D.171.ISIRR,172,D.156.DELTA
0,5508.632,3.491,18544
1,5423.467,2.482,19544
2,6365.863,3.864,23328
3,6768.402,7.132,24456
4,6921.315,10.439,25516
...,...,...,...
1150,7235.732,15.373,23344
1151,7166.721,13.519,23888
1152,7365.096,14.688,24096
1153,6501.263,14.699,21212


In [14]:
df.tail()

,timestamp,100,1000,1011,1017,1021,1027,1031,1037,104,...,V941,V951,V961,V978,V979,V981,V993,V994,V996,V998
1150,2025-05-11,26.290,1616901,13.138,29.654,13.893,26.080,15.542,31.179,12.923,...,3.005,3.005,3.005,313.300,0.044,85.571,315.258,0.048,78.930,84.076
1151,2025-05-12,431.185,1617540,13.447,30.346,13.756,27.253,14.117,31.883,55.639,...,2.938,2.938,2.938,311.708,0.102,85.956,307.940,0.104,77.841,82.178
1152,2025-05-13,1644.815,1618195,134.836,1243.556,177.001,1644.685,136.468,1245.111,177.740,...,2.943,2.943,2.943,327.996,0.179,NaN,331.664,0.171,81.579,83.941
1153,2025-05-14,26.755,1618766,11.887,30.632,14.683,27.160,11.936,32.436,12.842,...,2.992,2.992,2.992,291.945,0.276,88.747,292.173,0.273,NaN,86.690
1154,2025-05-15,25.043,1619500,17.274,30.104,16.257,27.117,15.866,32.436,17.231,...,2.916,2.916,2.916,367.545,0.305,86.986,369.531,0.300,80.168,84.738
